<a href="https://colab.research.google.com/github/sssteeefaaan/VIII-Semestar/blob/main/VIII-Semestar/Paralelni%20Sistemi/Laboratorijske%20Ve%C5%BEbe/Lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Instalacija
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

!git clone https://github.com/andreinechaev/nvcc4jupyter.git

!pip install ./nvcc4jupyter/

%load_ext nvcc_plugin

In [ ]:
#@title Zadatak 4
%%cu
#include <stdio.h>
#define N 512
#define BLOCK_SIZE 32

__host__ void sub_matrices_GPU(int* A, int* B, int* C);
__global__ void subtract_matrices(int* A, int* B, int* C);
__host__ bool check_result(int* A, int* B, int* GPU_C);
__device__ __host__ int subtract(int a, int b) { return a - b; }

int main(int argc, char** argv)
{
    int* A, *B, *GPU_C;
    A = (int*) malloc(sizeof(int) * N * N);
    B = (int*) malloc(sizeof(int) * N * N);
    GPU_C = (int*) malloc(sizeof(int) * N * N);
 
    for(int i = 0; i < N; i++) {
      for(int j = 0; j < N; j++) {
        A[i * N + j] = i * N + j;
        B[i * N + j] = j * N + i;
      }
    }
 
    sub_matrices_GPU(A, B, GPU_C);
 
    for(int i = 0; i < N; i++) {
        printf("|\t");
      for(int j = 0; j < N; j++)
        printf("%d\t", GPU_C[i * N + j]);
      printf("|\n");
    }
 
    if(check_result(A, B, GPU_C))
      printf("Matrice su identicne!\n");
    else
      printf("Matrice nisu identicne!\n");
 
    free(GPU_C);
    free(B);
    free(A);

    return 0;
}

__host__ bool check_result(int *A, int *B, int *GPU_C)
{
    bool rez = true;
    for(int i = 0; i < N && rez; i++)
      for(int j = 0; j < N && rez; j++)
        rez &= (GPU_C[i * N + j] == subtract(A[i * N + j], B[i * N + j]));
    return rez;
}

__host__ void sub_matrices_GPU(int *A, int *B, int *C)
{
    int* dev_A, *dev_B, *dev_C;
 
    cudaMalloc((void**) &dev_A, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_B, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_C, sizeof(int) * N * N);
 
    cudaMemcpy(dev_A, A, sizeof(int) * N * N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_B, B, sizeof(int) * N * N, cudaMemcpyHostToDevice);
 
    dim3 blockSize(BLOCK_SIZE, BLOCK_SIZE);
    dim3 gridSize(N / blockSize.x + 1, N / blockSize.y + 1);
 
    subtract_matrices<<<gridSize, blockSize>>>(dev_A, dev_B, dev_C);
 
    cudaMemcpy(C, dev_C, sizeof(int) * N * N, cudaMemcpyDeviceToHost);

    cudaFree(dev_A);
    cudaFree(dev_B);
    cudaFree(dev_C);
}

__global__ void subtract_matrices(int *A, int* B, int *C)
{
    int thread_id_x = blockIdx.x * blockDim.x + threadIdx.x;
    if(thread_id_x < N)
    {
        int thread_id_y = blockIdx.y * blockDim.y + threadIdx.y;
        if(thread_id_y < N)
          C[thread_id_x * N + thread_id_y] = subtract(A[thread_id_x * N + thread_id_y], B[thread_id_x * N + thread_id_y]);
    }

    //for(int i = thread_id_x; i < N; i+= blockDim.x)
    //  for(int j = thread_id_y; j < N; j += blockDim.y)
    //    C[i * N + j] = subtract(A[i * N + j], B[i * N + j]);
}

In [13]:
#@title Zadatak 7
%%cu
#include <stdio.h>
#define N 512
#define BLOCK_SIZE 32
#define GRID_SIZE 8

__host__ void saberi_matrice_na_GPU(int *A, int* B, int* C, int k, int p);
__global__ void saberi_matrice(int *A, int* B, int *C, __const__ int k, __const__ int p);
__host__ bool proveri_rezultat(int *A, int *B, int *Za_proveru, int k, int p);
__device__ __host__ int saberi_zameni(int a, int b, __const__ int k, __const__ int p);

int main(int argc, char** argv)
{
    int* A, *B, *C, k = 17190, p = 2904;
    A = (int*) malloc(sizeof(int) * N * N);
    B = (int*) malloc(sizeof(int) * N * N);
    C = (int*) malloc(sizeof(int) * N * N);
 
    for(int i = 0; i < N; i++) {
      for(int j = 0; j < N; j++) {
        A[i * N + j] = i * N + j;
        B[i * N + j] = j * N + i;
      }
    }
 
    saberi_matrice_na_GPU(A, B, C, k, p);

    for(int i = 0; i < N; i++) {
        printf("|\t");
      for(int j = 0; j < N; j++)
        printf("%d\t", C[i * N + j]);
      printf("|\n");
    }
 
    if(proveri_rezultat(A, B, C, k, p))
      printf("Rezultat je tacan!\n");
    else
      printf("Rezultat nije tacan!\n");
 
    free(C);
    free(B);
    free(A);

    return 0;
}

__host__ bool proveri_rezultat(int *A, int *B, int *Za_proveru, int k, int p)
{
    bool rez = true;
    for(int i = 0; i < N && rez; i++)
      for(int j = 0; j < N && rez; j++)
        rez &= (Za_proveru[i * N + j] == saberi_zameni(A[i * N + j], B[i * N + j], k, p));
    return rez;
}

__host__ void saberi_matrice_na_GPU(int *A, int *B, int *C, int k, int p)
{
    int* dev_A, *dev_B, *dev_C;
 
    cudaMalloc((void**) &dev_A, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_B, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_C, sizeof(int) * N * N);
 
    cudaMemcpy(dev_A, A, sizeof(int) * N * N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_B, B, sizeof(int) * N * N, cudaMemcpyHostToDevice);
 
    dim3 blockSize(BLOCK_SIZE, BLOCK_SIZE);
    dim3 gridSize(N / blockSize.x + 1, N / blockSize.y + 1);
 
    saberi_matrice<<<gridSize, blockSize>>>(dev_A, dev_B, dev_C, k, p);
 
    cudaMemcpy(C, dev_C, sizeof(int) * N * N, cudaMemcpyDeviceToHost);

    cudaFree(dev_A);
    cudaFree(dev_B);
    cudaFree(dev_C);
}

__global__ void saberi_matrice(int *A, int* B, int *C, __const__ int k, __const__ int p)
{
    int thread_id_x = blockIdx.x * blockDim.x + threadIdx.x;
    if(thread_id_x < N)
    {
        int thread_id_y = blockIdx.y * blockDim.y + threadIdx.y;
        if(thread_id_y < N)
          C[thread_id_x * N + thread_id_y] = saberi_zameni(A[thread_id_x * N + thread_id_y], B[thread_id_x * N + thread_id_y], k, p);
    }

    //for(int i = thread_id_x; i < N; i+= blockDim.x)
    //  for(int j = thread_id_y; j < N; j += blockDim.y)
    //    C[i * N + j] = saberi_zameni(A[i * N + j], B[i * N + j], k, p);
}

__device__ __host__ int saberi_zameni(int a, int b, __const__ int k, __const__ int p)
{
    int rez = a + b;
    if(rez > k)
      return p;
    return rez;
}

|	0	513	1026	1539	2052	2565	3078	3591	4104	4617	5130	5643	6156	6669	7182	7695	8208	8721	9234	9747	10260	10773	11286	11799	12312	12825	13338	13851	14364	14877	15390	15903	16416	16929	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	2904	290

In [ ]:
#@title Zadatak 8
%%cu
#include <stdio.h>
#define N 512
#define BLOCK_SIZE 32
#define GRID_SIZE 8

__host__ void oduzmi_matrice_na_GPU(int *A, int* B, int* C, int k, int p);
__global__ void oduzmi_matrice(int *A, int* B, int *C, __const__ int k, __const__ int p);
__host__ bool proveri_rezultat(int *A, int *B, int *Za_proveru, int k, int p);
__device__ __host__ int oduzmi_zameni(int a, int b, __const__ int k, __const__ int p);

int main(int argc, char** argv)
{
    int* A, *B, *C, k = 20, p = 666;
    A = (int*) malloc(sizeof(int) * N * N);
    B = (int*) malloc(sizeof(int) * N * N);
    C = (int*) malloc(sizeof(int) * N * N);
 
    for(int i = 0; i < N; i++) {
      for(int j = 0; j < N; j++) {
        A[i * N + j] = i * N + j;
        B[i * N + j] = j * N + i;
      }
    }
 
    oduzmi_matrice_na_GPU(A, B, C, k, p);

    for(int i = 0; i < N; i++) {
        printf("|\t");
      for(int j = 0; j < N; j++)
        printf("%d\t", C[i * N + j]);
      printf("|\n");
    }
 
    if(proveri_rezultat(A, B, C, k, p))
      printf("Rezultat je tacan!\n");
    else
      printf("Rezultat nije tacan!\n");
 
    free(C);
    free(B);
    free(A);

    return 0;
}

__host__ bool proveri_rezultat(int *A, int *B, int *Za_proveru, int k, int p)
{
    bool rez = true;
    for(int i = 0; i < N && rez; i++)
      for(int j = 0; j < N && rez; j++)
        rez &= (Za_proveru[i * N + j] == oduzmi_zameni(A[i * N + j], B[i * N + j], k, p));
    return rez;
}

__host__ void oduzmi_matrice_na_GPU(int *A, int *B, int *C, int k, int p)
{
    int* dev_A, *dev_B, *dev_C;
 
    cudaMalloc((void**) &dev_A, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_B, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_C, sizeof(int) * N * N);
 
    cudaMemcpy(dev_A, A, sizeof(int) * N * N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_B, B, sizeof(int) * N * N, cudaMemcpyHostToDevice);
 
    dim3 blockSize(BLOCK_SIZE, BLOCK_SIZE);
    dim3 gridSize(N / blockSize.x + 1, N / blockSize.y + 1);
 
    oduzmi_matrice<<<gridSize, blockSize>>>(dev_A, dev_B, dev_C, k, p);
 
    cudaMemcpy(C, dev_C, sizeof(int) * N * N, cudaMemcpyDeviceToHost);

    cudaFree(dev_A);
    cudaFree(dev_B);
    cudaFree(dev_C);
}

__global__ void oduzmi_matrice(int *A, int* B, int *C, __const__ int k, __const__ int p)
{
    int thread_id_x = blockIdx.x * blockDim.x + threadIdx.x;
    if(thread_id_x < N)
    {
        int thread_id_y = blockIdx.y * blockDim.y + threadIdx.y;
        if(thread_id_y < N)
          C[thread_id_x * N + thread_id_y] = oduzmi_zameni(A[thread_id_x * N + thread_id_y], B[thread_id_x * N + thread_id_y], k, p);
    }

    //for(int i = thread_id_x; i < N; i+= blockDim.x)
    //  for(int j = thread_id_y; j < N; j += blockDim.y)
    //    C[i * N + j] = oduzmi_zameni(A[i * N + j], B[i * N + j], k, p);
}

__device__ __host__ int oduzmi_zameni(int a, int b, __const__ int k, __const__ int p)
{
    int rez = a - b;
    if(rez < k)
      return p;
    return rez;
}

In [ ]:
#@title Zadatak 9
%%cu
#include <stdio.h>
#define N 512
#define BLOCK_SIZE 32
#define GRID_SIZE 8

__host__ void skaliraj_na_GPU(int *A, int x, int *C);
__global__ void sklairaj_matricu(int *A, __const__ int x, int * C);
__host__ bool proveri_rezultat(int *A, int x, int *Za_proveru);
__device__ __host__ int skaliraj(int a, int b) { return a * b; }

int main(int argc, char** argv)
{
    int* A, *C, x = 5;
    A = (int*) malloc(sizeof(int) * N * N);
    C = (int*) malloc(sizeof(int) * N * N);
 
    for(int i = 0; i < N; i++)
      for(int j = 0; j < N; j++)
        A[i * N + j] = j * N + i;
 
    skaliraj_na_GPU(A, x, C);

    for(int i = 0; i < N; i++) {
        printf("|\t");
      for(int j = 0; j < N; j++)
        printf("%d\t", C[i * N + j]);
      printf("|\n");
    }
 
    if(proveri_rezultat(A, x, C))
      printf("Rezultat je tacan!\n");
    else
      printf("Rezultat nije tacan!\n");
 
    free(C);
    free(A);

    return 0;
}

__host__ bool proveri_rezultat(int *A, int x, int *Za_proveru)
{
    bool rez = true;
    for(int i = 0; i < N && rez; i++)
      for(int j = 0; j < N && rez; j++)
        rez &= (Za_proveru[i * N + j] == skaliraj(A[i * N + j], x));
    return rez;
}

__host__ void skaliraj_na_GPU(int *A, __const__ int x, int *C)
{
    int* dev_A, *dev_C;
 
    cudaMalloc((void**) &dev_A, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_C, sizeof(int) * N * N);
 
    cudaMemcpy(dev_A, A, sizeof(int) * N * N, cudaMemcpyHostToDevice);
 
    dim3 blockSize(BLOCK_SIZE, BLOCK_SIZE);
    dim3 gridSize(N / blockSize.x + 1, N / blockSize.y + 1);
 
    sklairaj_matricu<<<gridSize, blockSize>>>(dev_A, x, dev_C);
 
    cudaMemcpy(C, dev_C, sizeof(int) * N * N, cudaMemcpyDeviceToHost);

    cudaFree(dev_A);
    cudaFree(dev_C);
}

__global__ void sklairaj_matricu(int *A, __const__ int x, int* C)
{
    int thread_id_x = blockIdx.x * blockDim.x + threadIdx.x;
    if(thread_id_x < N)
    {
        int thread_id_y = blockIdx.y * blockDim.y + threadIdx.y;
        if(thread_id_y < N)
          C[thread_id_x * N + thread_id_y] = skaliraj(A[thread_id_x * N + thread_id_y], x);
    }
        
    //for(int i = thread_id_x; i < N; i+= blockDim.x)
    //  for(int j = thread_id_y; j < N; j += blockDim.y)
    //    C[i * N + j] = skaliraj(A[i * N + j], x);
}

In [ ]:
#@title Zadatak 11
%%cu
#include <stdio.h>
#define N 512
#define BLOCK_SIZE 32
#define GRID_SIZE 8

__host__ void formiraj_na_GPU(int *a, int *b, int *c);
__global__ void formiraj_vektor(int *a, int *b, int *c);
__host__ bool proveri_rezultat(int *a, int *b, int *gpu);
__device__ __host__ int formiraj(int a, int b) { return max(a, b); }

int main(int argc, char** argv)
{
    int *a, *b, *c;
    a = (int*) malloc(sizeof(int) * N);
    b = (int*) malloc(sizeof(int) * N);
    c = (int*) malloc(sizeof(int) * N);
 
    for(int i = 0; i < N; i++)
      a[i] = N - (b[i] = i);
 
    formiraj_na_GPU(a, b, c);

    printf("c = |\t");
    for(int i = 0; i < N; i++)
      printf("%d\t", c[i]);
    printf("|\n");
 
    if(proveri_rezultat(a, b, c))
      printf("Rezultat je tacan!\n");
    else
      printf("Rezultat nije tacan!\n");
 
    free(c);
    free(b);
    free(a);

    return 0;
}

__host__ bool proveri_rezultat(int *a, int *b, int *gpu)
{
    bool rez = true;
    for(int i = 0; i < N && rez; i++)
        rez &= (gpu[i] == formiraj(a[i], b[i]));
    return rez;
}

__host__ void formiraj_na_GPU(int *a, int *b, int *c)
{
    int *dev_a, *dev_b, *dev_c;
 
    cudaMalloc((void**) &dev_a, sizeof(int) * N);
    cudaMalloc((void**) &dev_b, sizeof(int) * N);
    cudaMalloc((void**) &dev_c, sizeof(int) * N);
 
    cudaMemcpy(dev_a, a, sizeof(int) * N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, sizeof(int) * N, cudaMemcpyHostToDevice);
 
    formiraj_vektor<<<N / BLOCK_SIZE + 1, BLOCK_SIZE>>>(dev_a, dev_b, dev_c);
 
    cudaMemcpy(c, dev_c, sizeof(int) * N, cudaMemcpyDeviceToHost);

    cudaFree(dev_a);
    cudaFree(dev_b);
    cudaFree(dev_c);
}

__global__ void formiraj_vektor(int *a, int *b, int *c)
{
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    if(tid < N)
      c[tid] = formiraj(a[tid], b[tid]);
}